In [126]:
from lark import Lark, Transformer, v_args

In [186]:
mini_lisp_grammar = r"""
    # Grammar Rules
    ?program: stmt+
    ?stmt: print_stmt | def_stmt | exp
    ?print_stmt: "(print-num" exp ")" -> print_num
               | "(print-bool" exp ")" -> print_bool
    ?exp: bool_val | number | variable | num_op | logical_op | if_exp | fun_exp | fun_call
    ?num_op: plus | minus | multiply | divide | modulus | greater | smaller | equal
        plus: "(+" exp exp+ ")"
        minus: "(-" exp exp ")"
        multiply: "(*" exp exp+ ")"
        divide: "(/" exp exp ")"
        modulus: "(mod" exp exp ")"
        greater: "(>" exp exp ")"
        smaller: "(<" exp exp ")"
        equal: "(=" exp exp+ ")"
    ?logical_op: not_op | and_op | or_op
        and_op: "(and" exp exp+ ")"
        or_op: "(or" exp exp+ ")"
        not_op: "(not" exp ")"
    ?def_stmt: "(define" variable exp ")"
        variable: id
    ?fun_exp: "(fun" fun_ids fun_body ")"
        fun_ids: id*
        fun_body: exp
        ?fun_call: "(" fun_exp param* ")" | "(" fun_name param* ")"
        param: exp
        last_exp: exp
        fun_name: id
    ?if_exp: "(if" test_exp than_exp else_exp ")"
        test_exp: exp
        than_exp: exp
        else_exp: exp

    # Token Definitions
    number: /0|-?[1-9]\d*/
    id: /[a-z]([a-z0-9-])*/
    bool_val: /#t|#f/

    %import common.WS
    %ignore WS
"""
mini_lisp_parser = Lark(mini_lisp_grammar, start='program')

In [195]:
from lark import Transformer

class MiniLispTransformer(Transformer):
    def __init__(self):
        self.vars = {}

    def print_num(self, items):
        if type(items[0]) == int:
            print(items[0])
        else:
            print(self.vars[items[0]])
    def print_bool(self, items):
        # print(items[0])
        if (items[0]):
            print("#t")
        else:
            print("#f")

    def def_stmt(self, items):
        # print(items)
        var, val = items
        if var in self.vars:
            raise Exception("variable already defined")
        self.vars[var] = val

    def plus(self, items):
        return(sum(items))
    def minus(self, items):
        return(items[0] - items[1])
    def multiply(self, items):
        result = 1
        for i in items:
            result *= i
        return result
    def divide(self, items):
        # print(int(items[0] / items[1]))
        return(int(items[0] / items[1]))
    def modulus(self, items):
        # print(items[0] % items[1])
        return(items[0] % items[1])
    def greater(self, items):
        return items[0] > items[1]
    def smaller(self, items):
        return items[0] < items[1]
    def equal(self, items):
        for i in range(len(items)-1):
            if items[i] != items[i+1]:
                return False
        return True

    def and_op(self, items):
        # print("and:", items)
        for i in range(len(items)):
            if items[i] == False:
                return False
        return True
    def or_op(self, items):
        # print("or:", items)
        for i in range(len(items)):
            if items[i] == True:
                return True
        return False
    def not_op(self, items):
        # print("not:", items)
        return not items[0]

    def test_exp(self, items):
        return items[0]
    def than_exp(self, items):
        return items[0]
    def else_exp(self, items):
        return items[0]
    def if_exp(self, items):
        # print("if:", items)
        test, then, els = items
        if test:
            return then
        else:
            return els

    def variable(self, v):
        (v,) = v
        if v not in self.vars:
            return str(v)
        return self.vars[v]
    def number(self, n):
        (n,) = n
        return int(n)
    def id(self, i):
        (i,) = i
        return str(i)
    def bool_val(self, b):
        (b,) = b
        if b == "#t":
            return True
        return False

text = """
(define a (* 1 2 3 4))

(define b (+ 10 -5 -2 -1))

(print-num (+ a b))
"""

try:
    tree = mini_lisp_parser.parse(text)
    MiniLispTransformer().transform(tree)
except Exception as e:
    print("syntax error: ", e)
    # print("syntax error")

26


In [196]:
text = """
(define a (* 1 2 3 4))

(define b (+ 10 -5 -2 -1))

(print-num (+ a b))
"""
print(mini_lisp_parser.parse(text).pretty())

program
  def_stmt
    variable
      id	a
    multiply
      number	1
      number	2
      number	3
      number	4
  def_stmt
    variable
      id	b
    plus
      number	10
      number	-5
      number	-2
      number	-1
  print_num
    plus
      variable
        id	a
      variable
        id	b

